In [34]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import Select 
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd
from bs4 import BeautifulSoup
import json
import pandas as pd
import matplotlib.pyplot as plt
import koreanize_matplotlib
import folium
import numpy as np
import seaborn as sns

In [35]:
options = Options()
prefs = {'download.default_directory': '/Users/dw/ws/eda/data', 'download.prompt_for_download': False}
options.add_experimental_option('prefs', prefs)
service = Service("../driver/chromedriver-linux64/chromedriver")
driver = webdriver.Chrome(service=service, options=options)

# 주택보급률 신버젼 URL
url = "https://kosis.kr/statHtml/statHtml.do?orgId=408&tblId=DT_40803_N0002&lang_mode=ko&vw_cd=MT_ZTITLE&list_id=C14_001&conn_path=I4"
driver.get(url)
time.sleep(3)  # 페이지 로딩 대기


### 일단 driver를 이동해야하는 형태이다

In [36]:
# 첫 번째 iframe으로 전환
iframe1 = driver.find_element(By.XPATH, '//*[@id="iframe_rightMenu"]')
driver.switch_to.frame(iframe1)

# 두 번째 iframe으로 전환 (첫 번째 iframe 안에 있는 경우)
iframe2 = driver.find_element(By.XPATH, '//*[@id="iframe_centerMenu1"]')
driver.switch_to.frame(iframe2)

### 이동하고 나서 처음 창을 열때 기존에 클릭되어 있는걸 제거한다.

In [37]:
# 시점 버튼을 클릭한다.
button_time = driver.find_element(By.ID, "btn_time")
driver.execute_script("arguments[0].click();", button_time)
# 모든걸 해제한다.
uncheck_button = driver.find_element(By.ID, "timePopListMBtn")
driver.execute_script("arguments[0].click();", uncheck_button)
# 적용 버튼 클릭
#apply_button = driver.find_element(By.ID, "btnTimeAccept")
#driver.execute_script("arguments[0].click();", apply_button)

### 데이터를 가져오는 코드 최종본

In [38]:
import pandas as pd
from bs4 import BeautifulSoup
import time
from selenium import webdriver
from selenium.webdriver.common.by import By

# 연도와 월을 저장할 이중 리스트
months_to_check = []
for year in range(2003, 2025):  # 2024까지 포함
    if year == 2003:
        months_to_check.append([f"{year}.{month:02d}" for month in range(11, 13)])  # 2003.11, 2003.12
    elif year == 2024:
        months_to_check.append([f"{year}.{month:02d}" for month in range(1, 9)])  # 2024.01, ..., 2024.08
    else:
        months_to_check.append([f"{year}.{month:02d}" for month in range(1, 13)])  # 2004년부터 2023년까지

# 데이터프레임 초기화
data = []   # 수도권 아파트의 값을 저장할 리스트
data2 = []  # 서울 아파트의 값을 저장할 리스트
data3 = []  # 대구 아파트의 값을 저장할 리스트

# 각 연도별로 체크박스를 클릭
for year_index, year_months in enumerate(months_to_check):  # 모든 연도 처리
    for month in year_months:
        # month 값 출력
        print(f"Processing checkbox for: {month}")

        # 연도와 월을 조합하여 체크박스 찾기
        checkbox = driver.find_element(By.XPATH, f"//span[@class='fancytree-title' and text()='{month}']/..//span[contains(@class, 'fancytree-checkbox')]")
        # 체크박스 클릭
        driver.execute_script("arguments[0].click();", checkbox)

    # 적용 버튼 클릭
    apply_button = driver.find_element(By.ID, "btnTimeAccept")
    driver.execute_script("arguments[0].click();", apply_button)
    # 5초 대기
    time.sleep(5)

    # 시점 버튼 클릭
    button_time = driver.find_element(By.ID, "btn_time")
    driver.execute_script("arguments[0].click();", button_time)
    time.sleep(1)

    # 모든 체크박스 해제
    uncheck_button = driver.find_element(By.ID, "timePopListMBtn")
    driver.execute_script("arguments[0].click();", uncheck_button)

    # HTML 소스 가져오기
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    rows = soup.find_all('tr')

    # 수도권 데이터를 추출하는 반복문
    for row in rows:
        # "아파트"라는 title을 가진 <td> 찾기
        apartment_cell = row.find('td', title="아파트")
        if apartment_cell:
            # "수도권"이라는 title을 가진 다음 <td> 찾기
            region_cell = apartment_cell.find_next_sibling('td', title="수도권")
            if region_cell:
                # "value" 클래스를 가진 <td> 요소에서 값 추출
                values = [td.find('span', class_='val').text for td in row.find_all('td', class_='value')]
                if values:  # values 리스트가 비어있지 않을 때만 추가
                    data.extend(values)  # 수도권 값만 추가

    # 서울 데이터를 추출하는 반복문
    for row in rows:
        # "아파트"라는 title을 가진 <td> 찾기
        apartment_cell2 = row.find('td', title="아파트")
        if apartment_cell2:
            # "서울"이라는 title을 가진 다음 <td> 찾기
            region_cell2 = apartment_cell2.find_next_sibling('td', title="서울")
            if region_cell2:
                # "소계"가 3개인지 확인하기 위해 찾기
                subtotal_cells = []
                for _ in range(3):  # 소계가 3개인지를 확인
                    subtotal_cell = region_cell2.find_next_sibling('td', title="소계")
                    if subtotal_cell:
                        subtotal_cells.append(subtotal_cell)
                        region_cell2 = subtotal_cell  # 다음 소계를 찾기 위해 업데이트
                # 소계가 3개 모두 존재할 때
                if len(subtotal_cells) == 3:
                    # "value" 클래스를 가진 <td> 요소에서 값 추출
                    values2 = [td.find('span', class_='val').text for td in row.find_all('td', class_='value')]
                    if values2:  # values2 리스트가 비어있지 않을 때만 추가
                        data2.extend(values2)  # 서울 값만 추가

    # 대구 데이터를 추출하는 반복문
    for row in rows:
        # "아파트"라는 title을 가진 <td> 찾기
        apartment_cell3 = row.find('td', title="아파트")
        if apartment_cell3:
            # "대구"라는 title을 가진 다음 <td> 찾기
            region_cell3 = apartment_cell3.find_next_sibling('td', title="대구")
            if region_cell3:
                # "소계"가 3개인지 확인하기 위해 찾기
                subtotal_cells = []
                for _ in range(3):  # 소계가 3개인지를 확인
                    subtotal_cell = region_cell3.find_next_sibling('td', title="소계")
                    if subtotal_cell:
                        subtotal_cells.append(subtotal_cell)
                        region_cell3 = subtotal_cell  # 다음 소계를 찾기 위해 업데이트
                # 소계가 3개 모두 존재할 때
                if len(subtotal_cells) == 3:
                    # "value" 클래스를 가진 <td> 요소에서 값 추출
                    values3 = [td.find('span', class_='val').text for td in row.find_all('td', class_='value')]
                    if values3:  # values3 리스트가 비어있지 않을 때만 추가
                        data3.extend(values3)  # 대구 값만 추가

    # 1초 대기 후 다음 연도로 진행
    time.sleep(1)

# 데이터프레임 생성
if data or data2 or data3:  # data, data2 또는 data3 중 하나라도 데이터가 있을 때
    # months_to_check의 모든 월을 flatten하여 하나의 리스트로 만듭니다.
    month_labels = [month for year_months in months_to_check for month in year_months]

    # month_labels와 data를 zip하여 새로운 리스트 생성
    combined_data = list(zip(month_labels, data))
    combined_data2 = list(zip(month_labels, data2))  # 서울 아파트 데이터와 결합
    combined_data3 = list(zip(month_labels, data3))  # 대구 아파트 데이터와 결합

    # 데이터프레임 생성
    df = pd.DataFrame(combined_data, columns=['Year.Month', '수도권'])  # Year.Month와 수도권 값 포함
    df2 = pd.DataFrame(combined_data2, columns=['Year.Month', '서울'])  # 서울 아파트 값 추가
    df3 = pd.DataFrame(combined_data3, columns=['Year.Month', '대구'])  # 대구 아파트 값 추가

    # 데이터프레임 병합
    final_df = pd.merge(df, df2, on='Year.Month', how='outer')
    final_df = pd.merge(final_df, df3, on='Year.Month', how='outer')

    # 데이터프레임 출력
    print(final_df)
else:
    print("No data to display.")


Processing checkbox for: 2003.11
Processing checkbox for: 2003.12
Processing checkbox for: 2004.01
Processing checkbox for: 2004.02
Processing checkbox for: 2004.03
Processing checkbox for: 2004.04
Processing checkbox for: 2004.05
Processing checkbox for: 2004.06
Processing checkbox for: 2004.07
Processing checkbox for: 2004.08
Processing checkbox for: 2004.09
Processing checkbox for: 2004.10
Processing checkbox for: 2004.11
Processing checkbox for: 2004.12
Processing checkbox for: 2005.01
Processing checkbox for: 2005.02
Processing checkbox for: 2005.03
Processing checkbox for: 2005.04
Processing checkbox for: 2005.05
Processing checkbox for: 2005.06
Processing checkbox for: 2005.07
Processing checkbox for: 2005.08
Processing checkbox for: 2005.09
Processing checkbox for: 2005.10
Processing checkbox for: 2005.11
Processing checkbox for: 2005.12
Processing checkbox for: 2006.01
Processing checkbox for: 2006.02
Processing checkbox for: 2006.03
Processing checkbox for: 2006.04
Processing

### 대략 4분정도 걸린다.

### 값이 일치하는걸 확인했다.

### 오염방지를위해

In [39]:
final_df

,Year.Month,수도권,서울,대구
0,2003.11,45.9,46.7,49.1
1,2003.12,45.4,46.2,48.2
2,2004.01,45.2,46.0,47.9
3,2004.02,45.4,46.3,48.0
4,2004.03,45.6,46.5,48.2
...,...,...,...,...
245,2024.04,86.6,87.6,73.9
246,2024.05,87.0,87.9,73.7
247,2024.06,87.4,88.5,73.4
248,2024.07,87.9,89.2,73.2


In [40]:
final_df2 = final_df

In [41]:
# Year.Month을 datetime으로 변환
final_df2['Year.Month'] = pd.to_datetime(final_df2['Year.Month'], format='%Y.%m')
# 수도권 및 서울을 float으로 변환
final_df2['수도권'] = final_df2['수도권'].astype(float)
final_df2['서울'] = final_df2['서울'].astype(float)
final_df2['대구'] = final_df2['대구'].astype(float)
# 형변환 확인
print(final_df2.dtypes)


Year.Month    datetime64[ns]
수도권                  float64
서울                   float64
대구                   float64
dtype: object


In [42]:
final_df2

,Year.Month,수도권,서울,대구
0,2003-11-01,45.9,46.7,49.1
1,2003-12-01,45.4,46.2,48.2
2,2004-01-01,45.2,46.0,47.9
3,2004-02-01,45.4,46.3,48.0
4,2004-03-01,45.6,46.5,48.2
...,...,...,...,...
245,2024-04-01,86.6,87.6,73.9
246,2024-05-01,87.0,87.9,73.7
247,2024-06-01,87.4,88.5,73.4
248,2024-07-01,87.9,89.2,73.2


In [43]:
final_df2

,Year.Month,수도권,서울,대구
0,2003-11-01,45.9,46.7,49.1
1,2003-12-01,45.4,46.2,48.2
2,2004-01-01,45.2,46.0,47.9
3,2004-02-01,45.4,46.3,48.0
4,2004-03-01,45.6,46.5,48.2
...,...,...,...,...
245,2024-04-01,86.6,87.6,73.9
246,2024-05-01,87.0,87.9,73.7
247,2024-06-01,87.4,88.5,73.4
248,2024-07-01,87.9,89.2,73.2


In [ ]:
import mysql.connector
import pandas as pd

# connect_database 함수 정의
def connect_database(host_name="host_name", user_name="user_name", user_password="user_password"):
    mydb = mysql.connector.connect(
        host=host_name,
        port=3306,
        user=user_name,
        password=user_password,
        database="WAY_BACK_HOME"
    )
    cursor = mydb.cursor()
    return mydb, cursor

# 데이터베이스 연결
mydb, cursor = connect_database()

# 테이블 생성 (없을 경우)
create_table_query = """
CREATE TABLE IF NOT EXISTS HOUSE_PRICE_INDEX_APART (
    `Year_Month` DATE,
    `수도권` FLOAT,
    `서울` FLOAT,
    `대구` FLOAT
);
"""
cursor.execute(create_table_query)

# DataFrame final_df2를 SQL INSERT 문으로 변환하여 저장
for index, row in final_df2.iterrows():
    sql = "INSERT INTO HOUSE_PRICE_INDEX_APART  (`Year_Month`, `수도권`, `서울`, `대구`) VALUES (%s, %s, %s, %s)"
    cursor.execute(sql, (row['Year.Month'], row['수도권'], row['서울'], row['대구']))

# 변경 사항을 커밋하고 연결 종료
mydb.commit()
cursor.close()
mydb.close()
